In [ ]:
# MNIST preparation
import os
import torch
import torch.utils.data as data
import json
from PIL import Image
import torchvision.transforms.v2 as tfs



class DigitDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0
        self.files = []
        self.targets = torch.eye(10)

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))

    def __getitem__(self, item):
        path_file, target = self.files[item]
        t = self.targets[target]
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).ravel().float() / 255.0

        return img, t

    def __len__(self):
        return self.length


# Setting trnsform
to_tensor = tfs.ToImage()
d_dataset = DigitDataset("datasets/MNIST", transform=to_tensor)

# Loading data to dataset
train_data = data.DataLoader(d_dataset, batch_size=1, shuffle=True)

for i, tr in enumerate(train_data):
    if i==0:
        x, y = tr
        print(x.shape,y.shape)
    else:
        break

torch.Size([1, 784]) torch.Size([1, 10])


In [ ]:
# 1
import torch
import torch.nn as nn
import torch.utils.data as data

class LinnerudDataset(data.Dataset):
    def __init__(self):
        self.data_x = torch.tensor([[5.0, 162.0, 60.0], [2.0, 110.0, 60.0], [12.0, 101.0, 101.0], [12.0, 105.0, 37.0], [13.0, 155.0, 58.0], [4.0, 101.0, 42.0], [8.0, 101.0, 38.0], [6.0, 125.0, 40.0], [15.0, 200.0, 40.0], [17.0, 251.0, 250.0], [17.0, 120.0, 38.0], [13.0, 210.0, 115.0], [14.0, 215.0, 105.0], [1.0, 50.0, 50.0], [6.0, 70.0, 31.0], [12.0, 210.0, 120.0], [4.0, 60.0, 25.0], [11.0, 230.0, 80.0], [15.0, 225.0, 73.0], [2.0, 110.0, 43.0]])
        self.target = torch.tensor([[191.,  36.,  50.], [189.,  37.,  52.], [193.,  38.,  58.], [162.,  35.,  62.], [189.,  35.,  46.], [182.,  36.,  56.], [211.,  38.,  56.], [167.,  34.,  60.], [176.,  31.,  74.], [154.,  33.,  56.], [169.,  34.,  50.], [166.,  33.,  52.], [154.,  34.,  64.], [247.,  46.,  50.], [193.,  36.,  46.], [202.,  37.,  62.], [176.,  37.,  54.], [157.,  32.,  52.], [156.,  33.,  54.], [138.,  33.,  68.]])
        self.categories = ['Weight', 'Waist', 'Pulse']
        self.features = ['Chins', 'Situps', 'Jumps']
    
    def __getitem__(self, item):
        return self.data_x[item], self.target[item]
    
    def __len__(self):
        return len(self.data_x)
    

d_train = LinnerudDataset()
train_data = data.DataLoader(d_train, batch_size=8, shuffle=False)

it = iter(train_data)
x, y = next(it)
x, y = next(it)


In [5]:
# 2
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


class FuncDataset(data.Dataset):
    def __init__(self):
        _x = torch.arange(-5, 5, 0.1)
        self.data = _x
        self.target = torch.sin(2 * _x) + 0.2 * torch.cos(10 * _x) + 0.1 * _x**2  # значения функции в точках _x
        self.length = len(_x) # размер обучающей выборки

    def __getitem__(self, item):
        return self.data[item] , self.target[item]
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)

    def __len__(self):
        return self.length
        # возврат размера выборки


class FuncModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(3, 1)
        # модель однослойной полносвязной нейронной сети:
        # 1-й слой: число входов 3 (x, x^2, x^3), число нейронов 1

    def forward(self, x):
        xx = torch.empty(x.size(0), 3)
        xx[:, 0] = x
        xx[:, 1] = x ** 2
        xx[:, 2] = x ** 3
        y = self.layer1(xx)
        return y


torch.manual_seed(1)

# создать модель FuncModel
model = FuncModel()
model.train()
# перевести модель в режим обучения

epochs = 20 # число эпох обучения
batch_size = 8 # размер батча

d_train = FuncDataset() # создать объект класса FuncDataset
train_data = data.DataLoader(d_train, batch_size=batch_size, shuffle=True) # создать объект класса DataLoader с размером пакетов batch_size и перемешиванием образов выборки

optimizer = optim.Adam(params=model.parameters(), lr=0.01) # создать оптимизатор Adam для обучения модели с шагом обучения 0.01
loss_func = nn.MSELoss() # создать функцию потерь с помощью класса MSELoss

for _e in range(epochs): # итерации по эпохам
    for x_train, y_train in train_data:
        predict = model(x_train) # вычислить прогноз модели для данных x_train
        loss =  loss_func(predict, y_train.unsqueeze(-1))  # вычислить значение функции потерь

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# перевести модель в режим эксплуатации
model.eval()
# выполнить прогноз модели по всем данным выборки (d_train.data)
pred_all = model(d_train.data)
# вычислить потери с помощью loss_func по всем данным выборки; значение Q сохранить в виде вещественного числа

Q = loss_func(pred_all, d_train.target.unsqueeze(-1)).item()


In [ ]:
# 3
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

class FuncDataset(data.Dataset):
    def __init__(self):
        _range = torch.arange(-3, 3, 0.1)
        self.data = torch.tensor([(_x, _y) for _x in _range for _y in _range])
        self.target = self._func(self.data)
        self.length = len(self.data) # размер обучающей выборки

    @staticmethod
    def _func(coord):
        _x, _y = coord[:, 0], coord[:, 1]
        return torch.sin(2 * _x) * torch.cos(3 * _y) + 0.2 * torch.cos(10 * _x) * torch.sin(8 * _x) + 0.1 * _x ** 2 + 0.1 * _y ** 2

    def __getitem__(self, item):
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)
        return self.data[item], self.target[item]
        
    def __len__(self):
        # возврат размера выборки
        return self.length


class FuncModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(6,1)
        # модель однослойной полносвязной нейронной сети:
        # 1-й слой: число входов 6 (x, x^2, x^3, y, y^2, y^3), число нейронов 1

    def forward(self, coord):
        x, y = coord[:, 0], coord[:, 1]
        x.unsqueeze_(-1)
        y.unsqueeze_(-1)

        xx = torch.empty(coord.size(0), 6)
        xx[:, :3] = torch.cat([x, x ** 2, x ** 3], dim=1)
        xx[:, 3:] = torch.cat([y, y ** 2, y ** 3], dim=1)
        y = self.layer1(xx)
        return y

# здесь продолжайте программу
epochs = 20
batch_size = 16

model = FuncModel()

d_train = FuncDataset()
train_data = data.DataLoader(d_train, batch_size=batch_size, shuffle=True)

optimizer = optim.RMSprop(params=model.parameters(), lr = 0.01)
loss_func = nn.MSELoss()


# it = iter(train_data)
# next(it)

model.train()
for _e in range(epochs): # итерации по эпохам
    for coord, targ in train_data:
        predict = model(coord)
        loss = loss_func(predict, targ.unsqueeze(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# перевести модель в режим эксплуатации
model.eval()
# выполнить прогноз модели по всем данным выборки (d_train.data)
pred_all = model(d_train.data)
# вычислить потери с помощью loss_func по всем данным выборки; значение Q сохранить в виде вещественного числа

Q = loss_func(pred_all, d_train.target.unsqueeze(-1)).item()

In [ ]:
# 4 
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from sklearn.datasets import load_diabetes

# Загрузка датасета
diabetes = load_diabetes()

# Форма данных и целевой переменной
# print(diabetes.data.shape)  # (442, 10)
# print(diabetes.target.shape)  # (442,)

# Создание глобальных пееременных для тестирования
_global_var_data_x = torch.tensor(diabetes.data, dtype=torch.float32)
_global_var_target = torch.tensor(diabetes.target, dtype=torch.float32)

# здесь продолжайте программу
class DiabetesDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x
        self.target = _global_var_target.unsqueeze(1)
        self.length = len(self.data)

    def __getitem__(self, item):
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)
        return self.data[item], self.target[item]
        
    def __len__(self):
        # возврат размера выборки
        return self.length


class BloodShugarModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10,64)
        self.layer2 = nn.Linear(64,1)

    def forward(self, x):
        x = self.layer1(x)
        x = F.tanh(x)
        x = self.layer2(x)
        return x
    
epochs = 10
batch_size = 8
lr = 0.01

model = BloodShugarModel()
model.train()

optimizer = optim.RMSprop(params=model.parameters(), lr=lr)

d_train = DiabetesDataset()
train_data = data.DataLoader(d_train, batch_size=batch_size, shuffle=True)
loss_func = nn.MSELoss()


for _ in range(epochs):
    for data_x, targ in train_data:     
        predict = model(data_x)
        loss = loss_func(predict, targ)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()

pred_all = model(d_train.data)
Q = loss_func(pred_all, d_train.target).item()

In [ ]:
# 5
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


# здесь продолжайте программу
class MyDataset(data.Dataset):
    def __init__(self):
        _x = torch.arange(-6, 6, 0.1)
        self.data_x = _x
        self.target = 0.5 * self.data_x + torch.sin(2 * self.data_x) - 0.1 * torch.exp(self.data_x/2)  - 12.5
        self.lenght = len(_x)

    def __getitem__(self, item):
        return self.data_x[item], self.target[item]
    
    def __len__(self):
        return self.lenght
    

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(3, 1)

    def forward(self, x):
        xx = torch.empty(x.size(0), 3)
        xx[:, 0] = x
        xx[:, 1] = x ** 2
        xx[:, 2] = x ** 3
        y = self.layer1(xx)
        return y
    
model = MyModel()
model.train()

ep = 20
bs = 10

optimizer = optim.RMSprop(params=model.parameters(), lr=0.1)
loss_func = nn.MSELoss()

d_dataset = MyDataset()
train_data = data.DataLoader(d_dataset, batch_size=bs, shuffle=True)

for _ in range(ep):
    for data_x, targ in train_data:
        predict = model(data_x)
        loss = loss_func(predict, targ.unsqueeze(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()

pred_all = model(d_dataset.data_x)
Q = loss_func(pred_all ,d_dataset.target.unsqueeze(-1)).item()
